In [ ]:
%run PromptFunction

In [ ]:
from synapse.ml.core.platform import find_secret

# Fill in the following lines with your service information
service_name = "<MyOpenAIService>" # Name of your OpenAI service
deployment_name = "<MyModelDepl>" # Name of your deployment in OpenAI
key = find_secret("<MySecret>", "<MyKeyVault>")  # replace this with your secret and keyvault

NrOfReviews = 5 # Set number of Restaurantreviews


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col 

dfreviewid = df1 = spark.range(1,NrOfReviews + 1) \
    .withColumnRenamed("id", "reviewid") \
    .withColumn("prompt", restaurant_prompt_udf())

display(dfreviewid)



In [ ]:
from synapse.ml.cognitive import OpenAICompletion

OpenAICompletion = (
    OpenAICompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setUrl("https://{}.openai.azure.com/".format(service_name))
    .setMaxTokens(2048)
    .setPromptCol("prompt")
    .setErrorCol("error")
    .setOutputCol("response")
)

In [ ]:
from pyspark.sql.functions import col

dfreview = OpenAICompletion.transform(dfreviewid) \
    .select(col('reviewID'), col('response.choices.text').getItem(0).alias('reviewobject'))\
    .cache()

display(dfreview)

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, from_json

schema = StructType([ \
        StructField("restaurant", StringType(), False), \
        StructField("review", StringType(), False) \
        ])


df = dfreview.withColumn("json",from_json(col("reviewobject"), schema))\
    .select(col("reviewID"), col("json.*"))

display(df)

